# Transformers Part 2:

Text generation is the task of producing new text from a given input prompt. An example of text generation using machine learning is `GitHub’s Copilot`, which can generate code. Apart from code generation, text generation models can:

1. Generate stories, for example, try passing “Once upon a time " as input to a GPT-2 model.
2. Generate poetry.
3. Complete a paragraph with missing sentences. 
4. Summarize an article.
5. Answer questions given some context.

And many more...

In this notebook, we focus on training a causal language model from scratch. A causal language model tries to predict the next token in a given input sequence. In this way, it builds an understanding of the data it is trained on. 

So, the model in this notebook is __not finetuned__ but rather it is trained from scratch with randomly initialized parameters. 

__Agenda__:

1. Load the dataset from csv files
2. Tokenize and preprocess text
3. Create Batches
4. Train GPT-2 model 
5. Evaluate trained model

In [ ]:
!pip install datasets 
!pip install transformers
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.5 MB/s eta 0:00

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/nlp

/content/gdrive/My Drive/nlp


In [ ]:
!ls

ai-story_gen  dls.pkl  story_distilgpt2_finetune  story_gpt2_finetune
data	      models   story_gpt2


## Parameters for notebook execution:

We are going to execute this notebook from the terminal with the help of Papermill. So, it's better to store all the parameters need for a successful execution in one place. This way it's easy to manage the parameters. 

In [ ]:
import os 

# current directory path
DIR_PATH=os.getcwd()

In [ ]:
DIR_PATH

'/content/gdrive/My Drive/nlp'

In [ ]:
# parameters

# SAMPLE
TRAIN_ROWS=50000
TEST_ROWS=5000

# PATH OF CSV FILES
TRAIN_PATH= DIR_PATH+"/data/train_df.csv"
VALID_PATH=DIR_PATH+"/data/valid_df.csv"
TEST_PATH= DIR_PATH+"/data/test_df.csv"

# DATA PROCESSING
CONTEXT_LEN=256

# HYPERPARAMETERS
TRAIN_BS= 64
TEST_BS= 64 
EPOCHS=5

## Load the data:

The first step is to load the dataset from `CSV` files.We have to create the huggingface `Dataset` object after loading the data. 

In [ ]:
# import libraries
from datasets import load_dataset

In [ ]:
dataset = load_dataset("csv",
                       data_files={"train":TRAIN_PATH, "test": VALID_PATH})

#select a sample of dataset
dataset['train']= dataset['train'].select(range(TRAIN_ROWS))
dataset['test']=dataset['train'].select(range(TEST_ROWS))

dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-50b4a11504202c45/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'stories', 'prompts'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'stories', 'prompts'],
        num_rows: 5000
    })
})

In [ ]:
# remove unnamed column
dataset = dataset.remove_columns(['Unnamed: 0'])

dataset

DatasetDict({
    train: Dataset({
        features: ['stories', 'prompts'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['stories', 'prompts'],
        num_rows: 5000
    })
})

In [ ]:
import pickle 
# save dataset 
with open('tf_dataset.pkl', 'wb') as f:
    pickle.dump(dataset,f)

Let's see an example from the dataset

In [ ]:
for key in dataset["train"][0]:
    print(f"{key.upper()}: {dataset['train'][0][key][:500]} \n ")

STORIES: So many times have I walked on ruins , the remainings of places that I loved and got used to.. At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body.. After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly . <newline> It 's funny , b 
 
PROMPTS:  You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' 
 


We have two columns in our dataset:

1. stories
2. prompts

Since we are training a casual language model to generate stories, we want to train our model on the `stories` column in the dataset. Given the starting of a story, our model should complete the rest of the story.

## Data Processing:

The first step in the data processing phase is to tokenize the text. Since the input is going to small prompts, we can keep the context window small. This has the benefit that we can train the model much faster and it requires significantly less memory.  

Many documents in the dataset will have a size greater than the context window set by us. In this case, we break the documents into chunks with size equal to the context window. Sometimes, we also get rid of the last chunk if it is too small.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# loading a pretrained tokenizer for the gpt2 model 

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Let's see what the tokenizer does.

In [ ]:
outputs = tokenizer(
    dataset["train"][:2]["stories"],
    truncation=True,
    max_length=CONTEXT_LEN,
    return_overflowing_tokens=True,
    return_length=True,
)

In [ ]:
print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 7
Input chunk lengths: [256, 256, 240, 256, 256, 256, 129]
Chunk mapping: [0, 0, 0, 1, 1, 1, 1]


Now, let's create a tokenize function

In [ ]:
def tokenize(element):
    
    outputs = tokenizer(
        element["stories"],
        truncation=True,
        max_length=CONTEXT_LEN,
        return_overflowing_tokens=True,
        return_length=True,
    )
    
    input_batch = []
    
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == CONTEXT_LEN:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [ ]:
tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 114239
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 11363
    })
})

Ok!!! That was straightforward. We have completed the tokenization. Now, it is time to train the model. 

In [ ]:
# we can get our tokenized text back using the decode function 
tokenizer.decode(tokenized_datasets['train'][0]['input_ids'])

"So many times have I walked on ruins, the remainings of places that I loved and got used to.. At first I was scared, each time I could feel my city, my current generation collapse, break into the black hole that thrives within it, I could feel humanity, the way I'm able to feel my body.. After a few hundred years, the pattern became obvious, no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly. <newline> It's funny, but I felt as if after gaining what I desired so long, what I have lived for my entire life, only then, when I achieved immortality I started truly aging. <newline> <newline> 5 world wars have passed, and now they feel like a simple sickeness that would pass by every so often, I could no longer evaluate the individual human as a being of its own, the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced, a mechanism that is so dear

## Training the model

There are different transformer architectures that we can use:

1. Maybe a transformer from scratch
2. GPT-2
3. distilgpt2
4. facebook/opt-350m

And maybe, a encoder-decoder architecture like `BART`, `T5`

In this section, we are going to train a casual language model using the `GPT-2` transformer architecture. Later, we will focus on fine-tuning the hyperparameters for training and trying other architectures as well. 

In [ ]:
from transformers import GPT2LMHeadModel, AutoConfig

Let's initialize the model

In [ ]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=CONTEXT_LEN,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
# initialize the model
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())

# how many parameters?
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.4M parameters


### Creating batches:

HuggingFace provides us with the `DataCollatorForLanguageModeling` collator, which is designed specifically for language modeling. Besides stacking and padding batches, it also takes care of creating the language model labels — in causal language modeling the inputs serve as labels too (just shifted by one element), and this data collator creates them on the fly during training so we don’t need to duplicate the `input_ids`.

In [ ]:
from transformers import DataCollatorForLanguageModeling

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Inspect the data collator with an example

In [ ]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])

print("Collating data:")
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Collating data:
input_ids shape: torch.Size([5, 256])
attention_mask shape: torch.Size([5, 256])
labels shape: torch.Size([5, 256])


### Set training parameters:

In [ ]:
from transformers import Trainer, TrainingArguments
import math 

Let's set the hyperparameters for training.

In [ ]:
args = TrainingArguments(
    output_dir="ai-story_gen",
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=TEST_BS,
    evaluation_strategy="steps",
    eval_steps=1_50,
    logging_steps=1_50,
    gradient_accumulation_steps=8,
    num_train_epochs=EPOCHS,
    weight_decay=0.1,
    warmup_steps=1_00,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=False,
)

Initialize the `Trainer` API instance. 

In [ ]:
def compute_metrics(pred):
    # compute perplexity from the model's output
    perplexity = math.exp(-pred['eval_loss'])
    return {"perplexity": perplexity}

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    # compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
150,5.670500,4.496455
300,4.254900,3.972604
450,3.902100,3.706709
600,3.683300,3.534884
750,3.554600,3.420519
900,3.472600,3.349544
1050,3.396300,3.323036


TrainOutput(global_step=1115, training_loss=3.9558714451811237, metrics={'train_runtime': 2914.8915, 'train_samples_per_second': 195.958, 'train_steps_per_second': 0.383, 'total_flos': 7.4582675030016e+16, 'train_loss': 3.9558714451811237, 'epoch': 5.0})

In [ ]:
trainer.save_model("./story_gpt2")

## Story generator pipeline:

In [ ]:
import torch
from transformers import pipeline

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pipe = pipeline(
    "text-generation", model=model,tokenizer=tokenizer, device=device
)

Let's see an example of the text generated by our trained model

In [ ]:
txt = "Once upon a time there was a king and a queen."
print(pipe(txt, num_return_sequences=1,num_beams=3,
           do_sample=True, max_new_tokens=100,
          pad_token_id=tokenizer.eos_token_id)[0]["generated_text"])

Once upon a time there was a king and a queen. ” <newline> <newline> “ That ’ s what I ’ m looking for. ” <newline> <newline> “ What do you mean? ” <newline> <newline> “ I don ’ t know. It ’ s not like I ’ m going to be a king. ” <newline> <newline> “ It ’ s not like I ’ m a king


## Finally let's evaluate our language model:

The evaluation of a text generation task can be subjective and depend on the specific goals of the task. However, there are several common methods that can be used to evaluate the quality of the generated text.

1. __Human evaluation:__ One of the most reliable methods for evaluating text generation is through human evaluation. This involves having human evaluators read the generated text and rate its quality based on various metrics such as coherence, fluency, and relevance to the prompt or topic. This method provides a subjective evaluation of the text and can give insights into how well the generated text matches human writing.

2. __Automated metrics:__ Automated metrics can be used to evaluate text generation based on various quantitative measures. Examples of automated metrics include perplexity, BLEU score, and ROUGE score. These metrics compare the generated text to a reference text or corpus, and provide a measure of the similarity between the two texts.

3. __Domain-specific metrics:__ For text generation tasks that are focused on a specific domain, such as medical or legal writing, domain-specific metrics can be used to evaluate the quality of the generated text. These metrics are designed to measure the accuracy and completeness of the generated text within the specific domain.

4. __User testing:__ In some cases, user testing can be used to evaluate the quality of the generated text. This involves having users interact with the generated text and providing feedback on its quality, usefulness, and relevance to the task or topic.


In this notebook, we focus on evaluating the model based on automated metrics. We will check the model's performance based on three metrics:
- Perplexity: Here's how we can calculate perplexity
- BLEU score
- BERT score

In [ ]:
import torch
from tqdm import tqdm

from datasets import load_metric
from evaluate import load

__Perplexity__

In [ ]:
encodings = tokenizer("\n\n".join(dataset['test']['stories']),return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (3576967 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
encodings['input_ids'].shape

torch.Size([1, 3576967])

In [ ]:
max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0

for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

100%|█████████▉| 6985/6987 [03:04<00:00, 37.92it/s]


In [ ]:
ppl

tensor(110.9658, device='cuda:0')

__BLEU score__

In [ ]:
bleu_metric = load_metric('bleu')

<ipython-input-39-37a5b62391a6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')


In [ ]:
def data(n=10):
    for i in range(n):
        
        # pass first 10 words
        yield " ".join(dataset['test'][i]['stories'].split(" ")[:10]) 

In [ ]:
predictions=[]
references=[]

i=0
for out in pipe(data(),num_return_sequences=1,num_beams=3,
           do_sample=True, max_new_tokens=100,
          pad_token_id=tokenizer.eos_token_id):

    references.append(" ".join(dataset['test'][i]['stories'].split(" ")[10:100]))
    predictions.append(" ".join(out[0]['generated_text'].split(" ")[10:100]))
    
    i+=1


In [ ]:
preds=[i.split(" ") for i in predictions]
cands=[[i.split(" ")] for i in references]

bleu_scores = bleu_metric.compute(predictions=preds,
                                  references=cands)
print(bleu_scores)

{'bleu': 0.0, 'precisions': [0.258364312267658, 0.05303030303030303, 0.009652509652509652, 0.0], 'brevity_penalty': 0.5102459301400692, 'length_ratio': 0.5977777777777777, 'translation_length': 538, 'reference_length': 900}


__BERT score__

In [ ]:
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [ ]:
results['f1']

[0.8085444569587708,
 0.8051497340202332,
 0.8099966645240784,
 0.8321066498756409,
 0.8254857063293457,
 0.758840799331665,
 0.7905465364456177,
 0.8529961705207825,
 0.8341760039329529,
 0.8339055180549622]